In [3]:
import random
from firebase_admin import initialize_app, credentials, db

In [1]:
import json

In [4]:
cred = credentials.Certificate('creds.json')

In [5]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

In [6]:
seed = random.randrange(2 ** 32)
seed

3489313612

In [7]:
players = ["A", "B", "C"]
random.shuffle(players)

In [2]:
with open("board.json") as json_file:
    board = json.load(json_file)

In [8]:
db.reference("test/initial").set({
    'seed': seed,
    'players': players,
    'board': board,
})

In [ ]:

def bandit(row, col):

    board = board_ref.get()
    for r in range(row_count):
        for c in range(col_count):
            board[r][c]['bandit'] = r == row and c == col
    board_ref.set(board)
    
    for index, res_ref in enumerate(res_refs):
        resources = res_ref.get()
        res_count = len(resources)
        if len(resources) > 7:
            name = player_names[index]
            drop_count = res_count // 2
            print(f"{name} [{index}] must drop {drop_count} resources!")
    

def steal(from_index):

    resources = res_refs[from_index].get()
    steal_index = random.randrange(len(resources))
    resource = resources.pop(steal_index)
    res_refs[from_index].set(resources)
    
    give(current_index, [resource])
    
    
def knight(row, col, steal_index):
    
    cards = card_refs[current_index].get()
    cards.remove("Ritter")
    cards.append("Ritter (gespielt)")
    card_refs[current_index].set(cards)
    
    knight_count = cards.count("Ritter (gespielt)")
    turn = turn_ref.get()
    if knight_count > turn['maxKnightCount']:
        turn['maxKnightCount'] = knight_count
        turn['maxKnightIndex'] = current_index
        turn_ref.set(turn)
        
    bandit(row, col)
    steal(from_index)
    
    
def build(row1, col1, row2, col2):

    road(row1, col1)
    road(row2, col2)
    

def mono(res):

    gains = []
    
    for index, res_ref in enumerate(res_refs):
        if index == current_index:
            continue
        resources = res_ref.get()
        losses = [r for r in resources if r == res]
        take(index, losses)
        gains += losses
        
    give(current_index, gains)

        
def invent(res1, res2):
    
    give(current_index, [res1, res2])